# DeepLense Regression

A FastAI-based tool for performing regression on strong lensing images to predict axion mass density of galaxies.


In [ ]:
from fastai.basics import *
from fastai.vision.all import *
from fastai.callback.all import *
import torch
from torchvision import transforms

from models.xresnet_hybrid import xresnet_hybrid101
from utils.utils import inv_standardize,standardize, file_path, dir_path
from utils.custom_activation_functions import Mish_layer
from utils.custom_loss_functions import root_mean_squared_error, mae_loss_wgtd
from data.custom_datasets import RegressionNumpyArrayDataset

import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from tqdm import tqdm
import warnings

matplotlib.use('Agg')
%matplotlib inline
warnings.filterwarnings('ignore')

## Load the Data

In [ ]:
path_to_images = '/path_to_images.py'
path_to_masses = '/path_to_masses.py'
# Add mmap_mode='r' in np.load if you don't have enough RAM
images = np.load(path_to_images).astype('float32')
images = images.reshape(-1,1,150,150)
labels = np.load(path_to_masses).astype('float32')
labels = labels.reshape(-1,1)

In [ ]:
print(f'Shape of images: {images.shape}')
print(f'Shape of masses: {labels.shape}')

## Standardization

### Find stats

In [ ]:
IMAGES_MEAN, IMAGES_STD = images.mean(), images.std()
LABELS_MEAN, LABELS_STD = labels.mean(), labels.std()

### Standardize the dataset

In [ ]:
images=standardize(images,IMAGES_STD,IMAGES_MEAN)
labels=standardize(labels,LABELS_STD,LABELS_MEAN)

Now the mean values have to be close to 0, and the standard deviations are 1s, which will help the model to converge.

In [ ]:
print(f'Images mean, std: {images.mean()}, {images.std()}')
print(f'Labels mean, std: {labels.mean()}, {labels.std()}')

## Split the data

In [ ]:
np.random.seed(234)
num_of_images = labels.shape[0]
max_indx_of_train_images = int(num_of_images*0.85)
max_indx_of_valid_images = max_indx_of_train_images + int(num_of_images*0.1)
max_indx_num_of_test_images = max_indx_of_valid_images + int(num_of_images*0.05)
permutated_indx = np.random.permutation(num_of_images)
train_indx = permutated_indx[:max_indx_of_train_images]
valid_indx = permutated_indx[max_indx_of_train_images:max_indx_of_valid_images]
test_indx = permutated_indx[max_indx_of_valid_images:max_indx_num_of_test_images]

In [ ]:
print(f'Number of images in train: {int(num_of_images*0.85)}')
print(f'Number of images in valid: {int(num_of_images*0.1)}')
print(f'Number of images in test: {int(num_of_images*0.05)}')

## Transforms

In [ ]:
base_image_transforms = [
    transforms.Resize(150)
]
rotation_image_transofrms = [
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=(0,360))
]

## Create a Dataset

In [ ]:
train_dataset = RegressionNumpyArrayDataset(images, labels, train_indx,
                                            transforms.Compose(base_image_transforms+rotation_image_transofrms))
valid_dataset = RegressionNumpyArrayDataset(images, labels, valid_indx,                                          
                                            transforms.Compose(base_image_transforms))
test_dataset = RegressionNumpyArrayDataset(images, labels, test_indx,                                 
                                           transforms.Compose(base_image_transforms))

## Create a DataLoader

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')
batch_size = 64
dls = DataLoaders.from_dsets(train_dataset,valid_dataset,batch_size=batch_size, device=device, num_workers=2)

## Model Architecture

In [ ]:
torch.manual_seed(50)
model = xresnet_hybrid101(n_out=1, sa=True, act_cls=Mish_layer, c_in = 1,device=device)

## Create a Learner

In [ ]:
learn = Learner(
    dls, 
    model,
    opt_func=ranger, 
    loss_func= root_mean_squared_error,  
    metrics=[mae_loss_wgtd],
    model_dir = ''
)

## Find a Learning Rate

In [ ]:
learn.lr_find()

In [ ]:
num_of_epochs = 1
lr = 1e-2
learn.fit_one_cycle(num_of_epochs,lr,cbs=
                    [ShowGraphCallback,
                     SaveModelCallback(monitor='mae_loss_wgtd',fname='best_model')])

## Load the best model

In [ ]:
learn.load('best_model',device=device)
learn.model = learn.model.to(device)

## Get Predictions for the Test Dataset

In [ ]:
test_dl = DataLoader(test_dataset, batch_size=1,shuffle=False,device=device)
m_pred,m_true = learn.get_preds(dl=test_dl,reorder=False)

## Cancel standartization

In [ ]:
m_pred,m_true = inv_standardize(m_pred,LABELS_STD,LABELS_MEAN),inv_standardize(m_true,LABELS_STD,LABELS_MEAN)

## Plot the results

In [ ]:
test_mae = mae_loss_wgtd(m_pred,m_true)
plt.figure(figsize=(6,6),dpi=100)
plt.scatter(m_true, m_pred,  color='black')
line = np.linspace(0, 6, 100)
plt.plot(line, line)
plt.xlabel('Observed mass')
plt.ylabel('Predicted mass')
plt.text(1,4, 'MAE: {:.4f}'.format(test_mae))